<a href="https://colab.research.google.com/github/ArneHei/Backend_Mobility/blob/main/Backend_Mobility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install ipywidgets google-generativeai SQLAlchemy --quiet

In [ ]:
import google.generativeai as genai
import ipywidgets as widgets
from IPython.display import display, Markdown
import pandas as pd
from google.colab import userdata
from google.colab import files
from google.colab import drive
import datetime
import numpy as np
import sys
import os
import nbformat
import sqlalchemy
from sqlalchemy import create_engine, Column, Integer, String, Float, Boolean, Date, Time
from sqlalchemy.orm import sessionmaker, declarative_base

drive.mount('/content/drive')

# Define the file path to the 'Add shipments.ipynb' notebook
%run -i "/content/drive/MyDrive/Driver app/Add shipments.ipynb"
#%run -i "/content/drive/MyDrive/Driver app/Shipment class.ipynb"


GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
gemini_model = genai.GenerativeModel('models/gemini-2.5-pro')

Mounted at /content/drive


In [ ]:
#Load in SQL Database

# Define the base class for declarative models
Base = declarative_base()

class ShipmentDB(Base):
    __tablename__ = 'shipments'

    Shipment_ID = Column(String, primary_key=True) # Shipment_ID is now the primary key
    Transport = Column(String)
    Department = Column(String)
    Pickup_time = Column(String) # Storing as String to accommodate 'HH:MM-HH:MM' format
    Pickup_date = Column(Date)
    Delivery_time = Column(String) # Storing as String to accommodate 'HH:MM-HH:MM' format
    Delivery_date = Column(Date)
    Collection_Name = Column(String)
    Collection_City = Column(String)
    Collection_Address = Column(String)
    Collection_Postal_Code = Column(String)
    Delivery_Name = Column(String)
    Delivery_City = Column(String)
    Delivery_Address = Column(String)
    Delivery_Postal_Code = Column(String)
    Weight = Column(Float)
    Volume = Column(Float)
    Ldm = Column(Float)
    Content = Column(String)
    Units = Column(Integer)  # Store the integer value directly
    Unit_type = Column(String)
    Hazardous = Column(Boolean)

    def __repr__(self):
        return f"<Shipment(Shipment_ID='{self.Shipment_ID}', Transport='{self.Transport}', Department='{self.Department}')>"

# Create the engine to connect to the existing database file on Google Drive
db_path = '/content/drive/MyDrive/Driver app/shipments.db'
engine = create_engine(f'sqlite:///{db_path}')

# Create all tables in the engine.
Base.metadata.create_all(engine)

# Create a session to interact with the database
Session = sessionmaker(bind=engine)
session = Session()

print(f"SQLAlchemy base and ShipmentDB model defined. Database created/connected at {db_path}")
# Query all shipments from the database
all_shipments_from_db = session.query(ShipmentDB).all()

# Convert to a list of dictionaries for easier display or DataFrame creation
data_for_df = []
for s in all_shipments_from_db:
    data_for_df.append({
        'Shipment_ID': s.Shipment_ID, # Include Shipment_ID
        'Transport': s.Transport,
        'Department': s.Department,
        'Pickup_time': s.Pickup_time,
        'Pickup_date': s.Pickup_date,
        'Delivery_time': s.Delivery_time,
        'Delivery_date': s.Delivery_date,
        'Collection_Name': s.Collection_Name,
        'Collection_City': s.Collection_City,
        'Collection_Address': s.Collection_Address,
        'Collection_Postal_Code': s.Collection_Postal_Code,
        'Delivery_Name': s.Delivery_Name,
        'Delivery_City': s.Delivery_City,
        'Delivery_Address': s.Delivery_Address,
        'Delivery_Postal_Code': s.Delivery_Postal_Code,
        'Weight': s.Weight,
        'Volume': s.Volume,
        'Ldm': s.Ldm,
        'Content': s.Content,
        'Units': s.Units,
        'Unit_type': s.Unit_type,
        'Hazardous': s.Hazardous
    })

df_from_db = pd.DataFrame(data_for_df)

print(f"Successfully loaded {len(df_from_db)} shipments from the database.")
display(df_from_db.head())

# Close the session
session.close()

SQLAlchemy base and ShipmentDB model defined. Database created/connected at /content/drive/MyDrive/Driver app/shipments.db
Successfully loaded 2738 shipments from the database.


,Shipment_ID,Transport,Department,Pickup_time,Pickup_date,Delivery_time,Delivery_date,Collection_Name,Collection_City,Collection_Address,...,Delivery_City,Delivery_Address,Delivery_Postal_Code,Weight,Volume,Ldm,Content,Units,Unit_type,Hazardous
0,ARNSA-ATH53,None,ROXST,08:00-16:00,2025-08-01,00:01-23:59,2025-08-04,SONYA LEANAGE,UPPSALA,TRYFFELVÄGEN 2 LGH 1204 SE-756 46 UPPSALA,...,HÄSTVEDA,ÖSTRA STORGATAN 36 SE-282 75 HÄSTVEDA,282 75,50.0,0.01,0.0,TRÄNINGSMASKINER/TRÄ,1,CLL,False
1,ARNSA-ATH56,None,ROXST,08:00-16:00,2025-08-04,00:01-23:59,2025-08-05,KARL PERS,UPPSALA,SALAGATAN 22A SE-753 30 UPPSALA,...,TRELLEBORG,GODSVÄGEN 9 SE-231 62 TRELLEBORG,231 62,200.0,0.00,0.0,UNSPECIFIED GOODS,1,PLT,False
2,ARNSA-ATH59,None,ROXST,00:01-23:59,2025-08-07,00:01-23:59,2025-08-08,ES-MEDICAL AB,UPPSALA,LASTBILSGATAN 9 SE-754 54 UPPSALA,...,ARVIKA,FALLÄNGSVÄGEN 74 SE-671 51 ARVIKA,671 51,2.0,0.02,0.0,STBA 600,1,PCL,False
3,ARNSA-ATH64,None,ROXST,00:01-23:59,2025-08-05,00:01-23:59,2025-08-06,TOLLCO AB,KNIVSTA,RUBANKSGATAN 4 SE-741 71 KNIVSTA,...,HALMSTAD,DISTRIBUTION CENTER HALMSTAD MUSKÖTVÄGEN 3,302 62,2.0,0.00,0.0,VATTENSKYDD,1,PCL,False
4,ARNSA-ATH65,None,ROXST,00:01-23:59,2025-08-05,00:01-23:59,2025-08-06,TOLLCO AB,KNIVSTA,RUBANKSGATAN 4 SE-741 71 KNIVSTA,...,NÄSSJÖ,ANNE FORSVÄGEN 63 SE-571 36 NÄSSJÖ,571 36,1.0,0.00,0.0,VATTENSKYDD,1,PCL,False


In [ ]:
"""
file_paths = [
    "/content/drive/MyDrive/Colab Notebooks/Data handling/SEST0408.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/Data handling/SEST0508.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/Data handling/SEST0608.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/Data handling/SEST0708.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/Data handling/SEST0808.xlsx"
]

shipment_objects = process_shipment_files(file_paths)

print(f"Total shipment objects created: {len(shipment_objects)}")

# Displaying the first created shipment object for verification
if shipment_objects:
    print("\nDetails of the first created Shipment object:")
    first_shipment = shipment_objects[0]
    print(f"Shipment_ID: {first_shipment.Shipment_ID}")
    print(f"Transport: {first_shipment.Transport}")
    print(f"Department: {first_shipment.Department}")
    print(f"Pickup_time: {first_shipment.Pickup_time}")
    print(f"Pickup_date: {first_shipment.Pickup_date}")
    print(f"Delivery_time: {first_shipment.Delivery_time}")
    print(f"Delivery_date: {first_shipment.Delivery_date}")
    print(f"Collection_Name: {first_shipment.Collection_Name}")
    print(f"Units: {first_shipment.Units} {first_shipment.Unit_type} of {first_shipment.Content}")
    print("\n\nDetails of the 10th created Shipment object:")
    first_shipment = shipment_objects[9]
    print(f"Shipment_ID: {first_shipment.Shipment_ID}")
    print(f"Transport: {first_shipment.Transport}")
    print(f"Department: {first_shipment.Department}")
    print(f"Pickup_time: {first_shipment.Pickup_time}")
    print(f"Pickup_date: {first_shipment.Pickup_date}")
    print(f"Delivery_time: {first_shipment.Delivery_time}")
    print(f"Delivery_date: {first_shipment.Delivery_date}")
    print(f"Collection_Name: {first_shipment.Collection_Name}")
    print(f"Units: {first_shipment.Units} {first_shipment.Unit_type} of {first_shipment.Content}")
"""